In [445]:
%matplotlib inline
import numpy as np
from proj1_helpers import *
from implementations import *
from helpers import *
from cross_validation import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [446]:
y, features, ids = load_csv_data('train.csv')
yTest , featuresTest, idsTest = load_csv_data('test.csv')

In [447]:
index0_train = np.where(features[:,22] == 0)
index1_train = np.where(features[:,22] == 1)
index2_train = np.where((features[:,22] == 2) | (features[:,22] == 3))

index0_test = np.where(featuresTest[:,22] == 0)
index1_test = np.where(featuresTest[:,22] == 1)
index2_test = np.where((featuresTest[:,22] == 2) | (featuresTest[:,22] == 3))

In [448]:

def create_dataset(dataset, y, index0, index1, index2):
    
    jet_0 = dataset[index0]
    y0 = y[index0]
    
    jet_1 = dataset[index1]
    y1 = y[index1]
    
    jet_2_3 = dataset[index2]
    y2_3 = y[index2]
    

    jet_0 = np.delete(jet_0, [0, 4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28, 29], 1)
    jet_1 = np.delete(jet_1, [0, 4, 5, 6, 12, 22, 26, 27, 28], 1)
    jet_2_3 = np.delete(jet_2_3, [0, 22], 1)
    
    return jet_0, jet_1, jet_2_3, y0, y1, y2_3

In [449]:
jet0_train, jet1_train, jet2_3_train, y0_train, y1_train, y2_3_train = create_dataset(features, y, index0_train, index1_train, index2_train)
jet0_test, jet1_test, jet2_3_test, y0_test, y1_test, y2_3_test = create_dataset(featuresTest, yTest, index0_test, index1_test, index2_test)


In [450]:
def add_features_jet0(dataset):
    index_tau = [8, 9, 10]
    index_lep = [11, 12, 13]
    index_met = [14, 15, 16]
    
    features_to_add = []
   
     
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_lep))
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_met))
    
    
    features_to_add.append(compute_pseudo_rapidity(dataset, 9, 12))
    features_to_add.append(compute_pseudo_rapidity(dataset, 9, 15))
    features_to_add.append(compute_pseudo_rapidity(dataset, 12, 15)) 
    
    
    features_to_add = np.array(features_to_add).transpose()
    #features_to_add, mean, std = standardize(features_to_add)
    
    print(features_to_add.shape)
    return np.concatenate((dataset,features_to_add), 1)

In [451]:
jet0_train, mean, std = standardize(jet0_train)
jet0_test = (jet0_test - mean) / std

In [452]:
jet0_train = add_features_jet0(jet0_train)
jet0_test = add_features_jet0(jet0_test)

(99913, 5)
(227458, 5)


In [453]:
jet0_train = build_poly(jet0_train,1,7)
jet0_test = build_poly(jet0_test, 1, 7)

In [454]:
w0, loss0 = ridge_regression(y0_train, jet0_train,10**-5)
print(compute_rmse(y0_train, jet0_train, w0))
print(compute_rmse(y0_test, jet0_test, w0))

0.692640628263
1.87042222807


In [455]:
def add_features_jet1(dataset):
    index_tau = [8, 9, 10]
    index_lep = [11, 12, 13]
    index_met = [14, 15, 16]
    index_jet_lead = [17, 18, 19]
    
    
    features_to_add = []
   
    features_to_add.append(compute_invariant_mass(dataset, index_tau, index_jet_lead))
    features_to_add.append(compute_invariant_mass(dataset, index_lep, index_jet_lead))
    
    
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_lep))
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_met))
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_jet_lead))
    features_to_add.append(compute_transverse_mass(dataset, index_lep, index_jet_lead))
    features_to_add.append(compute_transverse_mass(dataset, index_jet_lead, index_met))
    
    
    
    features_to_add.append(compute_pseudo_rapidity(dataset, 9, 12))
    features_to_add.append(compute_pseudo_rapidity(dataset, 9, 15))
    features_to_add.append(compute_pseudo_rapidity(dataset, 9, 18)) 
    features_to_add.append(compute_pseudo_rapidity(dataset, 12, 15))
    features_to_add.append(compute_pseudo_rapidity(dataset, 12, 18))
    features_to_add.append(compute_pseudo_rapidity(dataset, 15, 18))
    
    features_to_add = np.array(features_to_add).transpose()
    #features_to_add, mean, std = standardize(features_to_add)
    
    print(features_to_add.shape)
    return np.concatenate((dataset,features_to_add), 1)


In [456]:
jet1_train, mean, std = standardize(jet1_train)
jet1_test = (jet1_test - mean) / std

In [457]:
jet1_train = add_features_jet1(jet1_train)

jet1_test = add_features_jet1(jet1_test)


(77544, 13)
(175338, 13)


In [458]:
jet1_train = build_poly(jet1_train,1,5)
jet1_test = build_poly(jet1_test, 1, 5)
print(jet1_train.shape, y1_train.shape)

(77544, 136) (77544,)


In [459]:
w1, loss1 = ridge_regression(y1_train, jet1_train, 10**-4)
print(compute_rmse(y1_train, jet1_train, w1))
print(compute_rmse(y1_test, jet1_test, w1))

0.807149943792
1.62809295189


In [460]:
def add_features_jet2_3(dataset):
    index_tau = [12, 13, 14]
    index_lep = [15, 16, 17]
    index_met = [18, 19, 20]
    index_jet_lead = [21, 22, 23]
    index_jet_sub = [24, 25, 26]
    
    features_to_add = []
   
    features_to_add.append(compute_invariant_mass(dataset, index_tau, index_jet_lead))
    features_to_add.append(compute_invariant_mass(dataset, index_tau, index_jet_sub))
    features_to_add.append(compute_invariant_mass(dataset, index_lep, index_jet_lead))
    features_to_add.append(compute_invariant_mass(dataset, index_lep, index_jet_sub))
    
    
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_lep))
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_met))
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_jet_lead))
    features_to_add.append(compute_transverse_mass(dataset, index_tau, index_jet_sub))
    features_to_add.append(compute_transverse_mass(dataset, index_lep, index_jet_lead))
    features_to_add.append(compute_transverse_mass(dataset, index_lep, index_jet_sub))
    features_to_add.append(compute_transverse_mass(dataset, index_jet_lead, index_jet_sub))
    features_to_add.append(compute_transverse_mass(dataset, index_jet_lead, index_met))
    features_to_add.append(compute_transverse_mass(dataset, index_jet_sub, index_met))
    
    
    features_to_add.append(compute_pseudo_rapidity(dataset, 13, 16))
    features_to_add.append(compute_pseudo_rapidity(dataset, 13, 22))
    features_to_add.append(compute_pseudo_rapidity(dataset, 13, 25)) 
    features_to_add.append(compute_pseudo_rapidity(dataset, 16, 22))
    features_to_add.append(compute_pseudo_rapidity(dataset, 16, 25))
    features_to_add.append(compute_pseudo_rapidity(dataset, 22, 25))
    
    features_to_add = np.array(features_to_add).transpose()
    #features_to_add, mean, std = standardize(features_to_add)
    
    print(features_to_add.shape)
    return np.concatenate((dataset,features_to_add), 1)
    

In [461]:
jet2_3_train, mean, std = standardize(jet2_3_train)
jet2_3_test = (jet2_3_test - mean) / std

In [462]:
jet2_3_train = add_features_jet2_3(jet2_3_train)
jet2_3_test = add_features_jet2_3(jet2_3_test)

(72543, 19)
(165442, 19)


In [463]:
jet2_3_train = build_poly(jet2_3_train,1,6)
jet2_3_test = build_poly(jet2_3_test, 1, 6)

In [464]:
w2, loss2 = ridge_regression(y2_3_train, jet2_3_train, 10**-5)
print(compute_rmse(y2_3_train, jet2_3_train, w2))
print(compute_rmse(y2_3_test, jet2_3_test, w2))

0.788116558255
1.60818395691


In [465]:
y0_predict = predict_labels(w0, jet0_test)
y1_predict = predict_labels(w1, jet1_test)
y2_predict = predict_labels(w2, jet2_3_test)

In [466]:
y_predict = np.zeros(len(yTest))
y_predict[index0_test] = y0_predict
y_predict[index1_test] = y1_predict
y_predict[index2_test] = y2_predict

In [467]:
create_csv_submission(idsTest, y_predict, 'prediction.csv')